In [1]:
import re
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup, SoupStrainer
import urllib
import os
from argparse import Namespace
from urllib.request import urlopen
import time

In [5]:
books = []
try:    
    for x in range(1,81):
        url = 'https://www.goodreads.com/list/show/3.Best_Science_Fiction_Fantasy_Books?page='+str(x)
        print(url)
        page = requests.get(url, allow_redirects=False, timeout=100)
        soup = BeautifulSoup(page.text, features="lxml")
        for i in soup.find_all("a", class_="bookTitle"):
            books.append(i)  
except Exception as e: 
    print(e)

https://www.goodreads.com/list/show/3.Best_Science_Fiction_Fantasy_Books?page=60
https://www.goodreads.com/list/show/3.Best_Science_Fiction_Fantasy_Books?page=61
https://www.goodreads.com/list/show/3.Best_Science_Fiction_Fantasy_Books?page=62
https://www.goodreads.com/list/show/3.Best_Science_Fiction_Fantasy_Books?page=63
https://www.goodreads.com/list/show/3.Best_Science_Fiction_Fantasy_Books?page=64
https://www.goodreads.com/list/show/3.Best_Science_Fiction_Fantasy_Books?page=65
https://www.goodreads.com/list/show/3.Best_Science_Fiction_Fantasy_Books?page=66
https://www.goodreads.com/list/show/3.Best_Science_Fiction_Fantasy_Books?page=67
https://www.goodreads.com/list/show/3.Best_Science_Fiction_Fantasy_Books?page=68
https://www.goodreads.com/list/show/3.Best_Science_Fiction_Fantasy_Books?page=69


In [6]:
x = 1
temp = []
review_urls = []

for book in books:
    metadata = None
    review = None
    i = 0
    while metadata is None:
        book_url= "https://www.goodreads.com" + book['href']
        try: 
            book_page = requests.get(book_url)
            metadata_soup = BeautifulSoup(book_page.content, features="lxml", parse_only=SoupStrainer('div', class_='BookPageMetadataSection'))
            metadata = metadata_soup.find('a', class_="RatingStatistics")
        except Exception as e: 
            print(e) 
            continue
        i = i+1
        if (i==30):
            break
    if (i==30):
        print("broke1")
        continue
    i = 0
    while review is None:
        review_soup = BeautifulSoup(book_page.content, features="lxml", parse_only=SoupStrainer('div', class_='ReviewsList'))
        review = review_soup.find('div', class_="Divider Divider--contents Divider--largeMargin")
        i = i+1
        if (i==30):
            break
    if (i==30):
        print("broke2")
        continue
    book_name = book.text.replace('\n', '')
    review_url = review.find('a')['href']
    author = metadata_soup.find('span', class_="ContributorLink__name")
    rating = metadata.find_all("div")[0].text.replace(u'\xa0','').replace(u'.',',')
    rate_count = metadata.find_all("div")[2].text.split("ratings")[0].replace(u'\xa0','')
    review_count = metadata.find_all("div")[2].text.split("ratings")[1].split("reviews")[0].replace(u'\xa0','')
    temp.append([book_name, author.text, rating, rate_count, review_count])
    review_urls.append([book_name, review_url])
    print(str(x), ", ", end="")
    x=x+1
    
temp2 = temp
review_urls2 = review_urls
print("done")

broke2
broke2
broke2
broke2
1 , 2 , 3 , 4 , broke2
5 , 6 , 7 , 8 , 9 , 10 , 11 , broke2
12 , 13 , 14 , 15 , 16 , 17 , 18 , 19 , 20 , 21 , broke2
22 , 23 , 24 , 25 , 26 , broke2
broke2
27 , broke2
broke2
28 , 29 , 30 , 31 , 32 , 33 , 34 , broke2
35 , 36 , 37 , 38 , broke2
39 , 40 , 41 , 42 , broke2
broke2
43 , 44 , 45 , 46 , 47 , 48 , 49 , broke2
50 , 51 , 52 , 53 , broke2
54 , 55 , broke2
56 , 57 , broke2
58 , broke2
broke2
broke2
59 , broke2
60 , 61 , broke2
62 , 63 , 64 , broke2
broke2
65 , 66 , 67 , 68 , 69 , 70 , 71 , 72 , 73 , 74 , 75 , broke2
broke2
76 , 77 , 78 , 79 , broke2
80 , 81 , 82 , 83 , broke2
84 , 85 , 86 , 87 , broke2
88 , 89 , 90 , 91 , 92 , 93 , 94 , broke2
95 , 96 , 97 , 98 , 99 , 100 , 101 , 102 , 103 , 104 , 105 , 106 , 107 , 108 , 109 , 110 , 111 , 112 , 113 , broke2
114 , 115 , broke2
broke2
broke2
broke2
116 , 117 , 118 , 119 , 120 , 121 , 122 , 123 , 124 , 125 , 126 , broke2
broke2
broke2
127 , 128 , 129 , 130 , broke2
131 , broke2
broke2
132 , broke2
133 , 13

In [7]:
books_df = pd.DataFrame(temp)
books_df.to_csv('books.csv', sep=';')

In [ ]:
reviews = []
x=0
for book in review_urls:
    print(str(x), ", ", end="")
    x=x+1
    book_reviews = []
    i = 0
    while not book_reviews:
        i = i+1
        book_url = "https://www.goodreads.com" + book[1]
        try:
            book_page = requests.get(book_url)
        except Exception as e: 
            print(e)  
            break
        book_soup = BeautifulSoup(book_page.content, features="lxml", parse_only=SoupStrainer('div', class_='ReviewsList'))
        book_reviews = book_soup.find_all('article', class_='ReviewCard') 
        if i==30:
            break
    if i==30:
        continue
    for r in book_reviews:
        try:
            reviewer_name = r.find('div', class_='ReviewerProfile__name').text
            reviewer_stats = r.find('div', class_='ReviewerProfile__meta').text
            review = r.find('span', class_='Formatted').text
            review_stats = r.find('footer', class_='SocialFooter').text
            if "Author" in reviewer_stats:
                reviewer_number = reviewer_stats.split("Author")[1].split("books")[0]
                reviewer_follower = reviewer_stats.split("books")[1].split("followers")[0]
            else: 
                reviewer_number = reviewer_stats.split(" reviews")[0]
                reviewer_follower = reviewer_stats.split(" reviews")[1].split("followers")[0]
            if "likes" in review_stats:
                review_like = review_stats.split(" likes")[0]
                review_comment = review_stats.split(" likes")[1].split("commentsLike")[0]
            else:
                review_like = 0
                review_comment = 0
        except Exception as e: 
            print(e)  
            continue
        reviews.append([book[0], reviewer_name, reviewer_number, reviewer_follower, review, review_like, review_comment])
print("done")
reviews2 = reviews

0 , 1 , list index out of range
2 , list index out of range
3 , list index out of range
4 , 5 , 6 , list index out of range
7 , list index out of range
8 , 9 , 10 , list index out of range
list index out of range
11 , list index out of range
12 , list index out of range
list index out of range
list index out of range
list index out of range
13 , 14 , 15 , 16 , list index out of range
17 , 18 , 19 , 20 , list index out of range
21 , 22 , list index out of range
23 , 24 , list index out of range
25 , list index out of range
26 , 27 , 28 , list index out of range
list index out of range
29 , 30 , 31 , list index out of range
32 , 33 , 34 , list index out of range
35 , 36 , list index out of range
37 , 38 , 39 , 40 , list index out of range
list index out of range
41 , 42 , list index out of range
43 , 44 , 45 , 46 , 47 , list index out of range
48 , 49 , list index out of range
50 , 51 , list index out of range
52 , list index out of range
53 , 54 , 55 , 56 , 57 , list index out of range


In [14]:
reviews_df = pd.DataFrame(reviews)
reviews_df.to_csv('reviews.csv', sep=';')